In [ ]:
import torch
from PIL import Image
import requests
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
import torchvision.transforms as T
torch.set_grad_enabled(False)

In [ ]:
# COCO classes
CLASSES = [
    'N/A', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A',
    'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse',
    'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack',
    'umbrella', 'N/A', 'N/A', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis',
    'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
    'skateboard', 'surfboard', 'tennis racket', 'bottle', 'N/A', 'wine glass',
    'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich',
    'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake',
    'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table', 'N/A',
    'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard',
    'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A',
    'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier',
    'toothbrush'
]

# colors for visualization
COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
          [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]]

In [ ]:
# for output bounding box post-processing
def box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=1)

def rescale_bboxes(out_bbox, size):
    img_w, img_h = size
    b = box_cxcywh_to_xyxy(out_bbox)
    b = b * torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32)
    return b

def plot_results(pil_img, prob, boxes):
    plt.figure(figsize=(16,10))
    plt.imshow(pil_img)
    ax = plt.gca()
    colors = COLORS * 100
    for p, (xmin, ymin, xmax, ymax), c in zip(prob, boxes.tolist(), colors):
        ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                   fill=False, color=c, linewidth=3))
        cl = p.argmax()
        text = f'{CLASSES[cl]}: {p[cl]:0.2f}'
        ax.text(xmin, ymin, text, fontsize=15,
                bbox=dict(facecolor='yellow', alpha=0.5))
    plt.axis('off')
    plt.show()

In [ ]:
model = torch.hub.load('facebookresearch/detr', 'detr_resnet50', pretrained=True)

In [ ]:
model.eval()

In [ ]:
url = 'http://images.cocodataset.org/val2017/000000039769.jpg'

In [ ]:
im = Image.open(requests.get(url, stream=True).raw)

In [ ]:
im

In [ ]:
transform = T.Compose([
  T.Resize(800),
  T.ToTensor(),
  T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
img = transform(im).unsqueeze(0) # unsqueeze

In [ ]:
outputs = model(img)

In [ ]:
outputs.keys()

In [ ]:
outputs['pred_logits'].shape

In [ ]:
print(outputs['pred_logits'].softmax(-1).shape)
print(outputs['pred_logits'].softmax(-1))

In [ ]:
probas = outputs['pred_logits'].softmax(-1)[0, :, :-1] # Why removing last element?

In [ ]:
keep = probas.max(-1).values > 0.9

In [ ]:
# convert boxes from [0; 1] to image scales
bboxes_scaled = rescale_bboxes(outputs['pred_boxes'][0, keep], im.size)

In [ ]:
plot_results(im, probas[keep], bboxes_scaled)

## Fine tuning DETR for licence plate detection
https://www.kaggle.com/code/nouamane/fine-tuning-detr-for-license-plates-detection/notebook

In [ ]:
pip install transformers

In [ ]:
pip install pytorch_lightning

In [ ]:
from transformers import DetrForObjectDetection
import pytorch_lightning as pl

In [ ]:
import torch

In [ ]:
class_names = [
  "Adidas", 
  "Apple", "BMW", "Citroen", "Cocacola",
  "DHL", "Fedex", "Ferrari", "Ford", "Google", 
  "HP", "Heineken", "Intel", "McDonalds", "Mini", 
  "Nbc", "Nike", "Pepsi", "Porsche", "Puma", 
  "RedBull", "Sprite", "Starbucks", "Texaco", "Unicef",
  "Vodafone", "Yahoo"
]

In [ ]:
# train_dataloder = torch.utils.data.DataLoader()
# val_dataloader = torch.utils.data.Dataloader()

In [ ]:
class Detr(pl.LightningModule):
  def __init__(self, lr, lr_backbone, weight_decay):
    super().__init__()
    # Replacel COCO classification head with custom head
    self.model = DetrForObjectDetection.from_pretrained(
      'facebook/detr-resnet-50', num_labels=len(class_names), ignore_mismatched_sizes=True
    )

    # see https://github.com/PyTorchLightning/pytorch-lightning/pull/1896
    self.lr = lr
    self.lr_backbone = lr_backbone
    self.weight_decay = weight_decay

  def forward(self, pixel_values, pixel_mask):
    outputs = self.model(pixel_values=pixel_values, pixel_mask=pixel_mask)
    return outputs

  def common_step(self, batch, batch_idx):
    pixel_values = batch['pixel_values']
    pixel_mask = batch['pixel_mask']
    labels = [{k:v.to(self.device) for k, v in t.items()} for t in batch['labels']]

    outputs = self.model(pixel_values, pixel_mask, labels)

    loss = outputs.loss
    loss_dict = outputs.loss_dict

    return loss, loss_dict

  def training_step(self, batch, batch_idx):
    loss, loss_dict = self.common_step(batch, batch_idx)
    self.log('training_loss', loss)
    for k, v in loss_dict.items():
      self.log('train_' + k, v.item())
    return loss

  def validation_step(self, batch, batch_idx):
    loss, loss_dict = self.common(batch, batch_idx)
    self.log('validation_loss', loss)
    for k, v in loss_dict.items():
      self.log('validation_' + k, v.item())
    return loss
  
  def configure_optimizers(self):
    param_dicts = [
      {'params': [p for n, p in self.named_paramters() if 'backbone' not in n and p.requires_grad]},
      {
       'params': [p for n, p in self.named_paramters() if 'backbone' in n and p.requires_grad],
       'lr': self.lr_backbone
      }
    ]
    optimizer = torch.optim.AdamW(param_dicts, lr=self.lr, weight_decay=self.weight_decay)
    return optimizer

  def train_dataloader(self):
    return train_dataloader

  def val_dataloader(self):
    return val_dataloader